In [ ]:
!pip install torch==1.8.1+cu111 torchvision==0.9.1+cu111 torchaudio===0.8.1 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip install transformers
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import requests
import re

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
import pandas as pd
import tweepy as tw
from tweepy import *

consumer_key = 'TUquBmNytxag3datK80q9UQyD'
consumer_secret = 'nXZgmIf2xQx5nkNzXPfVq0srgdqqzYcXtdAqNyJxHy1fBTot6Q'
access_key= '1333020132377935873-xq6b50ZDhT6s5rBcUPzQ2sicvU334A'
access_secret = 'H4hqsDjW8tkjGPhIUBtflmGKwzZnIU2ULQ1QCbwChe5Rj'

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tw.API(auth,wait_on_rate_limit=True)

search_words ="hotel" # you gonna search by hashtags , the "+" stands for concatenating keywords
new_search = search_words + " -filter:retweets"
number_posts = 15000 # choose the number of posts available in that day 

posts =[]

tweets = tw.Cursor(api.search,q=new_search,lang="en", since='2021-07-08', until='2021-08-03',count=50).items(number_posts)

for tweet in tweets:
    posts.append([tweet.created_at,tweet.text])


df = pd.DataFrame(posts)
df.to_csv('tweets.csv',index=False,header=["time","text"])

In [ ]:
import pandas as pd
tweet = pd.read_csv("../content/tweets.csv")

In [ ]:
import re
import string
# Data preprocessing
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [ ]:
tweet['text'] = tweet['text'].apply(lambda x:clean_text(x))

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk.tag import pos_tag
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
stop_words = stopwords.words("english")

def preprocess(text, lem=True):
    lemmatizer = WordNetLemmatizer()
    split_text = text.split()
    tokens = []
    if lem:
        for word, tag in pos_tag(split_text):
            if tag.startswith('NN'):
                pos = 'n'
            elif tag.startswith('VB'):
                pos = 'v'
            else:
                pos = 'a'
            tokens.append(lemmatizer.lemmatize(word, pos))
    else:
        tokens.append(split_text)
    return " ".join(tokens)

In [ ]:
tweet['text'] = tweet['text'].apply(lambda x:preprocess(x))

In [ ]:
import numpy as np
df = pd.DataFrame(np.array(tweet['text']), columns=['text'])

In [ ]:
def sentiment_score(review):
    tokens = tokenizer.encode(review, return_tensors='pt')
    result = model(tokens)
    return int(torch.argmax(result.logits))+1

In [ ]:
%%time
df['sentiment'] = df['text'].apply(lambda x: sentiment_score(x))

In [ ]:
df.to_csv('labelled_tweets_7.csv',index=False)

In [ ]:
import matplotlib.pyplot as plt
from collections import Counter

target_cnt = Counter(df.sentiment)

plt.figure(figsize=(16,8))
plt.bar(target_cnt.keys(), target_cnt.values())
plt.title("Dataset labels distribuition")

In [ ]:
tweet

,time,text
0,2021-08-02 23:59:51,@PinkRantz @MysterySolvent He has a lot of bus...
1,2021-08-02 23:59:44,I'm not sure how describing your Times Square ...
2,2021-08-02 23:59:41,So @SouthwestAir rebooked my flight which is a...
3,2021-08-02 23:59:33,legit so hot in here plus my slight nausea... ...
4,2021-08-02 23:59:19,@ChairmanWabbit the hotel i work at is better
...,...,...
14996,2021-08-02 05:42:05,This new boutique hotel promises Kasauli minus...
14997,2021-08-02 05:42:03,"Imma book a hotel, just meet me there"
14998,2021-08-02 05:41:56,Victorians can now access Pfizer regardless of...
14999,2021-08-02 05:41:50,[SNS UPDATE]\n\n ☆ Abrus Kad...
